In [2]:
!pip3 install chromadb
!pip3 install langchain "unstructured[pdf]" sentence_transformers


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Load the Documents

Here we use Langchain to load all documents from our `./docs` directory. More info on how this works can be found on [Langhchain's website](https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory)

In [3]:
from langchain.document_loaders import DirectoryLoader
directory = './docs'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

### Chunking the Documents

Here we chunk the document's contents from which embeddings will be generated. You can experiment with the `chunk_size` and `chunk_overlap` to fit your use case.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

8


### Choose Your Embedding Model

Here we're using `BAAI/bge-small-en` as our model, but this can be swapped out with any one of the BGE models from the [BGE Model List](https://huggingface.co/BAAI/bge-large-en)

In [5]:
# Example of using Setence Transformers instead of BGE models
# from langchain.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name="instructor-large")

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/Users/garethharte/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Vectors into Chroma DB

Here we load the vector embeddings from our documents into the [Chroma DB](https://docs.trychroma.com/getting-started) vector store. Note this could easily be swapped out with something like Pinecone for higher throughput.

In [6]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

len(db.get()['ids'])




8

### Query the Vector DB

Here we query the locally running instance of Chroma DB. By default we want to return the top 3 results. Increase `k=3` if you would like to return more than 3 of the top results.

In [7]:
query = "How many employees does wilmington tutoring program have?"
matching_docs = db.similarity_search(query,k=3)

matching_docs

[Document(page_content='Jessica Hollowitz — Business Manager responsible for free consultations (sales calls), tutor matching, administration, tutor coordination, etc. Based in Wilmington but working remotely.\n\nJea — personal assistant to the owner, based in the Philippines\n\nLegal Structure\n\nWilmington Tutoring Program LLC is a North Carolina LLC. It will be acquired by a to-be-formed NewCo (also a NC LLC).\n\nMarket Research\n\nIndustry\n\nThe practice of private tutoring long precedes that of formal education.\n\nTo this day, the industry remains highly fragmented and relatively unorganized. It comprises of individual tutors, learning centers (often franchised), online tutoring services and online tutor matching portals. The total number of tutors is unknown as it remains in many ways a cottage industry. Concerns about child development have been trending higher despite efforts to improve schools and learning.', metadata={'source': 'docs/bizplan.pdf'}),
 Document(page_content='